In [1]:
import tensorflow as tf
import numpy as np

In [2]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [3]:
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

In [4]:
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

In [5]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)
        
    return input_batch, target_batch

In [6]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_step = 3
n_input = n_class = dic_len

In [21]:
tf.reset_default_graph()
X = tf.placeholder(dtype=tf.float32, shape=[None, n_step, n_input])
Y = tf.placeholder(dtype=tf.int32, shape=[None])

In [22]:
W = tf.Variable(tf.truncated_normal([n_hidden, n_class]))
b = tf.Variable(tf.truncated_normal([n_class]))

In [23]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

In [24]:
_, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

In [25]:
model = tf.matmul(states[1].h, W) + b

In [26]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))

In [27]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [28]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
    
    print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.6f}'.format(loss))
    
print('optimization finished!')

Epoch: 0001 cost = 3.144093
Epoch: 0002 cost = 2.368072
Epoch: 0003 cost = 1.529271
Epoch: 0004 cost = 1.297672
Epoch: 0005 cost = 0.695091
Epoch: 0006 cost = 0.846541
Epoch: 0007 cost = 0.513233
Epoch: 0008 cost = 0.353555
Epoch: 0009 cost = 0.440567
Epoch: 0010 cost = 0.433114
Epoch: 0011 cost = 0.497065
Epoch: 0012 cost = 0.371867
Epoch: 0013 cost = 0.208256
Epoch: 0014 cost = 0.330867
Epoch: 0015 cost = 0.166439
Epoch: 0016 cost = 0.198430
Epoch: 0017 cost = 0.059590
Epoch: 0018 cost = 0.097875
Epoch: 0019 cost = 0.038223
Epoch: 0020 cost = 0.077588
Epoch: 0021 cost = 0.110167
Epoch: 0022 cost = 0.037522
Epoch: 0023 cost = 0.031314
Epoch: 0024 cost = 0.039640
Epoch: 0025 cost = 0.137348
Epoch: 0026 cost = 0.008191
Epoch: 0027 cost = 0.022710
Epoch: 0028 cost = 0.007737
Epoch: 0029 cost = 0.031598
Epoch: 0030 cost = 0.047349
optimization finished!


In [29]:
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X: input_batch, Y: target_batch})

predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)
    
print('input value:', [w[:3] + ' ' for w in seq_data])
print('predicted value:', predict_words)
print('accuracy:', accuracy_val)

input value: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
predicted value: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
accuracy: 1.0
